In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/ultralytics/yolov5  

!pip install -qr yolov5/requirements.txt  
%cd yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 12622, done.
remote: Counting objects: 100% (155/155), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 12622 (delta 100), reused 105 (delta 65), pack-reused 12467
Receiving objects: 100% (12622/12622), 12.83 MiB | 33.77 MiB/s, done.
Resolving deltas: 100% (8694/8694), done.
     |████████████████████████████████| 1.6 MB 35.4 MB/s 
/content/yolov5


In [6]:
import torch
import yaml
from IPython.display import Image
from IPython.core.magic import register_line_cell_magic

In [5]:
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 1.12.1+cu113 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)


In [12]:
%cd /content
!curl -L "https://app.roboflow.com/ds/HLhKZCkMyC?key=qFT8Vn62hf" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

/content
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   901  100   901    0     0    589      0  0:00:01  0:00:01 --:--:--   588
100 50.7M  100 50.7M    0     0  18.3M      0  0:00:02  0:00:02 --:--:-- 62.1M
Archive:  roboflow.zip
 extracting: README.dataset.txt      
 extracting: README.roboflow.txt     
 extracting: data.yaml               
   creating: test/
   creating: test/images/
 extracting: test/images/image_1040_jpg.rf.2c59d4a25b22ce306f1389901695449f.jpg  
 extracting: test/images/image_1131_jpg.rf.de511e0454def9a246dfb3610ccced9c.jpg  
 extracting: test/images/image_1140_jpg.rf.e24d3b816f740d113bc5b84c13e32d79.jpg  
 extracting: test/images/image_1169_jpg.rf.7da6d6af950a1c0e22dbdc9536dc1e5a.jpg  
 extracting: test/images/image_1172_jpg.rf.9fb5461a8e8d1c4e339a457c79dd17a3.jpg  
 extracting: test/images/image_1187_jpg.rf.bafb50af7f7506ab5a263506ff6a7842.jpg  
 

In [13]:
# define number of classes based on YAML
with open("data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

print(num_classes)

11


In [14]:
%cat /content/yolov5/models/yolov5s.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: 80  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]]

In [15]:
%cat /content/yolov5/setup.cfg

# Project-wide configuration file, can be used for package metadata and other toll configurations
# Example usage: global configuration for PEP8 (via flake8) setting or default pytest arguments
# Local usage: pip install pre-commit, pre-commit run --all-files

[metadata]
license_file = LICENSE
description_file = README.md


[tool:pytest]
norecursedirs =
    .git
    dist
    build
addopts =
    --doctest-modules
    --durations=25
    --color=yes


[flake8]
max-line-length = 120
exclude = .tox,*.egg,build,temp
select = E,W,F
doctests = True
verbose = 2
# https://pep8.readthedocs.io/en/latest/intro.html#error-codes
format = pylint
# see: https://www.flake8rules.com/
ignore =
    E731  # Do not assign a lambda expression, use a def
    F405  # name may be undefined, or defined from star imports: module
    E402  # module level import not at top of file
    F401  # module imported but unused
    W504  # line break after binary operator
    E127  # continuation line over-indented for visua

In [16]:
#customize iPython writefile so we can write variables
@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [20]:
# train yolov5s on custom data for 400 epochs
%%time
%cd /content/yolov5/
!python train.py --img 416 --batch 32 --epochs 400 --data '../data.yaml' --cfg ./models/yolov5s.yaml --weights '' --name FinalExp --cache

/content/yolov5
train: weights=, cfg=./models/yolov5s.yaml, data=../data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=400, batch_size=32, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=FinalExp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-152-gdb68474 Python-3.7.14 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hs

In [22]:
%cp /content/yolov5/runs/train/FinalExp/weights/best.pt /content/drive/MyDrive/Colab\ Notebooks/STEAM/Final\ project/

In [24]:
%cd /content/yolov5/
!python detect.py --weights runs/train/FinalExp//weights/best.pt --img 416 --conf 0.1 --source /content/test/images/image_1972_jpg.rf.05c630be3eefd2c35168edaa959f2869.jpg

/content/yolov5
detect: weights=['runs/train/FinalExp//weights/best.pt'], source=/content/test/images/image_1972_jpg.rf.05c630be3eefd2c35168edaa959f2869.jpg, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v6.2-152-gdb68474 Python-3.7.14 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7039792 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /content/test/images/image_1972_jpg.rf.05c630be3eefd2c35168edaa959f2869.jpg: 416x416 2 Cars, 1 No Stop Sign, 8.2ms
Speed: 0.4ms pre-process, 8.2ms inference, 1.5ms NMS per image at shape (1, 3, 416, 416)
Results saved to runs/detect/exp2
